In [1]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import time
import json
import matplotlib.pyplot as plt

We are going to read the energy eigenvalues and dipole matrix elements from .npy files.  We will also still create an instance of the PFHamiltonianGenerator class so we can use its build_pcqed_pf_hamiltonian() method, but it is not really important what details we use to instantiate this class... so we will use LiH in a minimal basis since this is a fast way to instantiate the class!

In [2]:
# # Set the molecule and bond length
# mol = psi4.geometry("""
# 0 1
# O
# H 1 1.0
# """)

# # Optimize the geometry to obtain the new coordinates
# psi4.optimize('scf/cc-pvdz')

# # Get the new optimized coordinates
# new_coordinates = mol.geometry()

# # Print the new coordinates
# print(new_coordinates)

In [3]:
# read data from .npy files for formaldehyde casci(8,8) calculations

# !!! Change this to the correct path on your computer!
npy_folder = "/Users/jfoley19/Code/data_repository/Mapol/LiH/PCQED/6-311G/r_1.4/"

# these file names should still be good
E_npy_file = npy_folder + "LiH_r_1.4_6311g_fci_Energies.npy"
Mu_npy_file = npy_folder + "LiH_r_1.4_6311g_fci_Dipoles.npy"

# store energy eigenvalues in E_array
E_array = np.load(E_npy_file)
# store dipole matrix elements in Mu_array
Mu_array = np.load(Mu_npy_file)

# print their shape so we know how many elements we have
print(np.shape(E_array))
print(np.shape(Mu_array))
print(E_array)

(250,)
(250, 250, 3)
[-8.01219476 -7.899962   -7.88593596 -7.86104863 -7.86104863 -7.84851087
 -7.84851087 -7.80693549 -7.78459936 -7.76702524 -7.76342939 -7.75827098
 -7.75827098 -7.75123644 -7.75123644 -7.72171149 -7.68303148 -7.53167595
 -7.46183148 -7.44561152 -7.44561152 -7.44122102 -7.41700688 -7.41700688
 -7.41499491 -7.41499491 -7.40506676 -7.40302238 -7.39081209 -7.38221021
 -7.38221021 -7.37219855 -7.37219855 -7.36897222 -7.36897222 -7.35609042
 -7.35609042 -7.33714534 -7.33343267 -7.32130747 -7.31448363 -7.31448363
 -7.306799   -7.306799   -7.30583178 -7.30363511 -7.29983049 -7.29983049
 -7.29710106 -7.28736609 -7.28736609 -7.27939388 -7.27254896 -7.26716866
 -7.26189291 -7.26189291 -7.26028229 -7.26028229 -7.24463063 -7.24419397
 -7.24419397 -7.24111323 -7.23094221 -7.22795844 -7.22795844 -7.2276023
 -7.2276023  -7.21883619 -7.21553431 -7.20683738 -7.20683738 -7.19644038
 -7.18180272 -7.17656251 -7.15921998 -7.15921998 -7.15824942 -7.15347534
 -7.15347534 -7.14947376 -7.149

In [4]:
# setup basic arguments to create an instance of the PFHamiltonianGenerator class
mol_str = """
    Li
    H 1 1.4
    symmetry c1
"""


options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}


cavity_free_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 10, 
}

# create the instance of our PFHamiltonianGenerator class
instance = PFHamiltonianGenerator(mol_str, options_dict, cavity_free_dict)


Start SCF iterations:

Canonical RHF One-electron energy = -12.6946346236295895
CQED-RHF One-electron energy      = -12.6912246781500926
Nuclear repulsion energy          = 1.1339511657214287
Dipole energy                     = 0.0032271304319229
SCF Iteration   1: Energy = -7.8595725022589402   dE = -7.85957E+00   dRMS = 8.01325E-05
SCF Iteration   2: Energy = -7.8595728148569677   dE = -3.12598E-07   dRMS = 9.74325E-06
SCF Iteration   3: Energy = -7.8595728258670885   dE = -1.10101E-08   dRMS = 3.56735E-06
SCF Iteration   4: Energy = -7.8595728281752013   dE = -2.30811E-09   dRMS = 4.53151E-08
SCF Iteration   5: Energy = -7.8595728281752510   dE = -4.97380E-14   dRMS = 1.23024E-10
SCF Iteration   6: Energy = -7.8595728281752510   dE =  0.00000E+00   dRMS = 8.29882E-12
Total time for SCF iterations: 0.001 seconds 

QED-RHF   energy: -7.86053866 hartree
Psi4  SCF energy: -7.86053866 hartree
 Completed QED-RHF in 0.22067928314208984 seconds
 Completed 1HSO Build in 5.412101745605469e-0

In [5]:
N_R = 21
d_array = np.linspace(0, 20, N_R)
N_l = len(d_array)
N_el = 50
N_ph = 10
omega = 0.12086
lambda_vector = np.array([0, 0, 0.0])

# create an array of zeros to store the PCQED eigenvalues for each value of d
_pcqed_22 = np.zeros((N_l, N_el * N_ph))
# loop over values of d, build Hamiltonian, capture eigenvalues
ctr = 0
for d in d_array:
    instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector , E_array, Mu_array, neglect_DSE=False)      # E_array[:,ctr]: 20 energy values for fisrt displacement and so on... 
    _pcqed_22[ctr, :] = instance.PCQED_pf_eigs
    ctr += 1

In [6]:
# E_pcqed_22 = instance.PCQED_pf_eigs
print(f"Ground state{_pcqed_22[0,0]:.12f}")
# print(_pcqed_22)

Ground state-8.012194758187
